<a href="https://colab.research.google.com/github/BelajarMachineLearning/Pengenalan-Deep-Learning/blob/main/4_Optimization_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

4423680/4422102 [==============================] - 0s 0us/step


# Batch Normalization

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

In [ ]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.8440 - accuracy: 0.7153 - val_loss: 0.5529 - val_accuracy: 0.8148
Epoch 2/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.5754 - accuracy: 0.7997 - val_loss: 0.4809 - val_accuracy: 0.8380
Epoch 3/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.5209 - accuracy: 0.8195 - val_loss: 0.4502 - val_accuracy: 0.8506
Epoch 4/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.4816 - accuracy: 0.8314 - val_loss: 0.4259 - val_accuracy: 0.8558
Epoch 5/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.4604 - accuracy: 0.8392 - val_loss: 0.4112 - val_accuracy: 0.8594
Epoch 6/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.4393 - accuracy: 0.8455 - val_loss: 0.4000 - val_accuracy: 0.8626
Epoch 7/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.4261 - accuracy: 0.8509 - val_loss: 0.3909 - val_accuracy

# Reusing Pretrained Layers

In [ ]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))
    
(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]  

In [ ]:
X_train_A.shape

(43986, 28, 28)

In [ ]:
X_train_B.shape

(200, 28, 28)

In [ ]:
y_train_A[:30]

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [ ]:
y_train_B[:30]

array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)

Train on Model A (multi-class classification task)

In [ ]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="relu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [ ]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 5s 4ms/step - loss: 1.8278 - accuracy: 0.3686 - val_loss: 1.3511 - val_accuracy: 0.6129
Epoch 2/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.9249 - accuracy: 0.7593 - val_loss: 0.6575 - val_accuracy: 0.8169
Epoch 3/20
1375/1375 [==============================] - 5s 3ms/step - loss: 0.5575 - accuracy: 0.8339 - val_loss: 0.4753 - val_accuracy: 0.8460
Epoch 4/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.4467 - accuracy: 0.8525 - val_loss: 0.4167 - val_accuracy: 0.8558
Epoch 5/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.4013 - accuracy: 0.8629 - val_loss: 0.3826 - val_accuracy: 0.8657
Epoch 6/20
1375/1375 [==============================] - 5s 3ms/step - loss: 0.3771 - accuracy: 0.8697 - val_loss: 0.3632 - val_accuracy: 0.8739
Epoch 7/20
1375/1375 [==============================] - 5s 3ms/step - loss: 0.3612 - accuracy: 0.8756 - val_loss: 0.3478 - val_accuracy:

In [ ]:
model_A.save("model_A.h5")

Train on Model B (binary classification) from scratch


In [ ]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="relu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 1s 45ms/step - loss: 0.7069 - accuracy: 0.5000 - val_loss: 0.7062 - val_accuracy: 0.4625
Epoch 2/20
7/7 [==============================] - 0s 14ms/step - loss: 0.7007 - accuracy: 0.5050 - val_loss: 0.7003 - val_accuracy: 0.4807
Epoch 3/20
7/7 [==============================] - 0s 14ms/step - loss: 0.6948 - accuracy: 0.5300 - val_loss: 0.6943 - val_accuracy: 0.5091
Epoch 4/20
7/7 [==============================] - 0s 16ms/step - loss: 0.6890 - accuracy: 0.5650 - val_loss: 0.6895 - val_accuracy: 0.5568
Epoch 5/20
7/7 [==============================] - 0s 13ms/step - loss: 0.6842 - accuracy: 0.6450 - val_loss: 0.6843 - val_accuracy: 0.6298
Epoch 6/20
7/7 [==============================] - 0s 14ms/step - loss: 0.6791 - accuracy: 0.7400 - val_loss: 0.6795 - val_accuracy: 0.6998
Epoch 7/20
7/7 [==============================] - 0s 17ms/step - loss: 0.6744 - accuracy: 0.7750 - val_loss: 0.6753 - val_accuracy: 0.7485
Epoch 8/20
7/7 [===========

In [ ]:
model_B.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 300)               235500    
_________________________________________________________________
dense_10 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_11 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_12 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_13 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_14 (Dense)             (None, 1)                

Use the pre-trained Model A

In [ ]:
model_A = keras.models.load_model("model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [ ]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False #freeze

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

In [ ]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True #unfreeze

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 0s 18ms/step - loss: 0.1752 - accuracy: 0.9950 - val_loss: 0.1878 - val_accuracy: 0.9838
Epoch 2/4
7/7 [==============================] - 0s 16ms/step - loss: 0.1673 - accuracy: 0.9950 - val_loss: 0.1807 - val_accuracy: 0.9848
Epoch 3/4
7/7 [==============================] - 0s 13ms/step - loss: 0.1602 - accuracy: 0.9950 - val_loss: 0.1744 - val_accuracy: 0.9848
Epoch 4/4
7/7 [==============================] - 0s 13ms/step - loss: 0.1537 - accuracy: 0.9950 - val_loss: 0.1684 - val_accuracy: 0.9848
Epoch 1/16
7/7 [==============================] - 1s 72ms/step - loss: 0.1479 - accuracy: 0.9950 - val_loss: 0.1625 - val_accuracy: 0.9848
Epoch 2/16
7/7 [==============================] - 0s 17ms/step - loss: 0.1419 - accuracy: 0.9950 - val_loss: 0.1572 - val_accuracy: 0.9848
Epoch 3/16
7/7 [==============================] - 0s 13ms/step - loss: 0.1363 - accuracy: 0.9950 - val_loss: 0.1518 - val_accuracy: 0.9858
Epoch 4/16
7/7 [===============